In [1]:
import pandas as pd
import numpy as np

In [2]:
machine_1 = {
    "pieces_per_hour": 144,
    "defect_rate": "2%",
    "cycle_time_sec": 55,
    "hours_to_mantainaince":34,
    "improvement_percentage": "16.67%"
}

machine_2 = {
    "pieces_per_hour": 103,
    "defect_rate": "1.5%",
    "cycle_time_sec": 35,
    "hours_to_mantainaince":3.7,
    "improvement_percentage": "60%",
}

In [ ]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpStatus

# Definir datos de las máquinas
maquinas_data = {
    "maquina": ["M1", "M2", "M3", "M4"],
    "tipo": ["dual", "dual", "simple", "simple"],
    "capacidad_por_hora": [144, 144, 105, 105],  # Capacidad de piezas por hora
    "cambio_producto_A": [2.1, 2.1, None, None],  # Tiempo de setup para la  producción para el producto A en minutos
    "cambio_producto_B": [1.75, 1.75, 2.1, 2.1]   # Tiempo de setup para la producción para el producto B en minutos
}

# Crear DataFrame de máquinas
df_maquinas = pd.DataFrame(maquinas_data)

In [105]:
df_maquinas

,maquina,tipo,capacidad_por_hora,cambio_producto_A,cambio_producto_B
0,M1,dual,144,2.4,1.75
1,M2,dual,144,2.4,1.75
2,M3,simple,105,NaN,2.10
3,M4,simple,105,NaN,2.10


In [7]:
df_pedidos = pd.read_csv("./new_orders/orders.csv")

In [5]:
df_pedidos = df_pedidos.rename({'order_id':'pedido_id', 'product':'producto', 'quantity':'cantidad', 'priority':'prioridad'})

In [8]:
modelo = LpProblem("Optimización de Producción con Múltiples Pedidos", LpMinimize)

# Variables de decisión: asignación de cada pedido a cada máquina
produccion = {}
for _, maquina in df_maquinas.iterrows():
    for _, pedido in df_pedidos.iterrows():
        if not pd.isna(maquina[f'cambio_producto_{pedido["producto"]}']):
            produccion[(maquina["maquina"], pedido["pedido_id"])] = LpVariable(
                f"produccion_{maquina['maquina']}_pedido_{pedido['pedido_id']}", 0, pedido["cantidad"], cat="Integer"
            )

# Función objetivo: minimizar el tiempo total de producción ponderado por la prioridad y equilibrar carga
objetivo = []
for (maquina, pedido_id), variable in produccion.items():
    pedido = df_pedidos.loc[df_pedidos["pedido_id"] == pedido_id].iloc[0]
    producto = pedido["producto"]
    tiempo_produccion = df_maquinas.loc[df_maquinas["maquina"] == maquina, f'cambio_producto_{producto}'].values[0]
    prioridad = pedido["prioridad"]
    ponderacion_prioridad = 1 / prioridad  # Prioridad alta reduce el tiempo en la función objetivo
    objetivo.append(tiempo_produccion * ponderacion_prioridad * variable)

modelo += lpSum(objetivo), "Tiempo Total Ponderado por Prioridad"

# Restricciones
# 1. Satisfacer la demanda de cada pedido
for _, pedido in df_pedidos.iterrows():
    pedido_id = pedido["pedido_id"]
    modelo += lpSum(produccion.get((maquina, pedido_id), 0) for maquina in df_maquinas["maquina"]) == pedido["cantidad"], f"demanda_pedido_{pedido_id}"

# 2. Limitar la capacidad máxima de producción por hora para cada máquina
for _, maquina in df_maquinas.iterrows():
    max_capacidad = maquina["capacidad_por_hora"]
    tiempo_total = lpSum(produccion.get((maquina["maquina"], pedido_id), 0) for pedido_id in df_pedidos["pedido_id"])
    modelo += tiempo_total <= max_capacidad, f"capacidad_{maquina['maquina']}"

# 3. Limitar el tiempo total de operación de cada máquina a 8 horas (480 minutos) y equilibrar carga
max_tiempo_maquina = 480  # en minutos
for _, maquina in df_maquinas.iterrows():
    total_tiempo_maquina = lpSum(produccion.get((maquina["maquina"], pedido_id), 0) *
                                 df_maquinas.loc[df_maquinas["maquina"] == maquina["maquina"], 
                                 f'cambio_producto_{df_pedidos.loc[df_pedidos["pedido_id"] == pedido_id]["producto"].values[0]}'].values[0]
                                 for pedido_id in df_pedidos["pedido_id"] 
                                 if (maquina["maquina"], pedido_id) in produccion)
    
    modelo += total_tiempo_maquina <= max_tiempo_maquina, f"tiempo_total_maquina_{maquina['maquina']}"

# Resolver el modelo
modelo.solve()

# Mostrar resultados
print("Asignación de pedidos a máquinas:")
for (maquina, pedido_id), variable in produccion.items():
    if variable.varValue > 0:
        print(f"Máquina {maquina} produce {variable.varValue} unidades del Pedido {pedido_id}")
print(f"Tiempo Total Ponderado por Prioridad: {modelo.objective.value()}")
print(f"Estado de la optimización: {LpStatus[modelo.status]}")

/home/facu/Documents/github/Freudenberg_challenge/venv/lib/python3.10/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


KeyError: 'producto'

In [122]:
# Inicializar lista para almacenar los resultados
resultados = []

# Recorrer los resultados del modelo y añadir las filas al DataFrame
for (maquina, pedido_id), variable in produccion.items():
    if variable.varValue > 0:  # Solo incluimos asignaciones mayores a 0
        resultados.append({
            "maquina": maquina,
            "pedido_id": pedido_id,
            "unidades_producidas": variable.varValue
        })

# Crear DataFrame con los resultados
df_resultados = pd.DataFrame(resultados)

# Mostrar el DataFrame
print(df_resultados)

   maquina  pedido_id  unidades_producidas
0       M1          4                 42.0
1       M1         16                 48.0
2       M1         24                 45.0
3       M1         25                 48.0
4       M2          1                 42.0
5       M2          5                 41.0
6       M2         14                 46.0
7       M2         40                 46.0
8       M3          8                 45.0
9       M3         20                 41.0
10      M3         23                 49.0
11      M3         44                 45.0
12      M3         50                115.0
13      M4         11                 60.0
14      M4         31                 45.0


In [95]:
modelo.solve()

# Crear listas para almacenar los resultados
resultados = {
    "maquina": [],
    "pedido_id": [],
    "producto": [],
    "cantidad_producida": [],
    "prioridad": [],
    "tiempo_produccion_minutos": [],
}

# Recorrer las variables de decisión y extraer los resultados
for (maquina, pedido_id), variable in produccion.items():
    if variable.varValue > 0:  # Solo consideramos los casos donde se produce algo
        # Obtener datos del pedido
        pedido = df_pedidos.loc[df_pedidos["pedido_id"] == pedido_id].iloc[0]
        producto = pedido["producto"]
        cantidad = variable.varValue
        prioridad = pedido["prioridad"]

        # Obtener el tiempo unitario de producción en minutos para la máquina y el producto
        tiempo_unitario = df_maquinas.loc[df_maquinas["maquina"] == maquina, f'cambio_producto_{producto}'].values[0]
        tiempo_total = tiempo_unitario * cantidad

        # Almacenar en el diccionario de resultados
        resultados["maquina"].append(maquina)
        resultados["pedido_id"].append(pedido_id)
        resultados["producto"].append(producto)
        resultados["cantidad_producida"].append(cantidad)
        resultados["prioridad"].append(prioridad)
        resultados["tiempo_produccion_minutos"].append(tiempo_total)

# Convertir el diccionario de resultados a un DataFrame de pandas
df_resultados = pd.DataFrame(resultados)

# Mostrar el DataFrame de resultados
print(df_resultados)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/silvestf/Documents/freudenberg/venv/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/5e7a66cb7f3f452899f6ac5dbb02ca3b-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/5e7a66cb7f3f452899f6ac5dbb02ca3b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 63 COLUMNS
At line 1024 RHS
At line 1083 BOUNDS
At line 1244 ENDATA
Problem MODEL has 58 rows, 160 columns and 480 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

   maquina  pedido_id producto  cantidad_producida  prioridad  \
0       M1          4        A                42.0          1   
1       M1         16        A                48.0          2   
2       M1       

In [90]:
# resultados = []

# # Recorrer los resultados del modelo y añadir filas con tiempos al DataFrame
# for (maquina, pedido_id), variable in produccion.items():
#     if variable.varValue > 0:  # Solo incluimos asignaciones mayores a 0
#         # Obtener información del pedido y de la máquina
#         pedido = df_pedidos.loc[df_pedidos["pedido_id"] == pedido_id].iloc[0]
#         producto = pedido["producto"]
#         tiempo_produccion_unitario = df_maquinas.loc[df_maquinas["maquina"] == maquina, f'cambio_producto_{producto}'].values[0]
        
#         # Calcular el tiempo total de producción para la asignación
#         tiempo_total_produccion = tiempo_produccion_unitario * variable.varValue

#         # Agregar los resultados al DataFrame
#         resultados.append({
#             "maquina": maquina,
#             "pedido_id": pedido_id,
#             "producto": producto,
#             "unidades_producidas": variable.varValue,
#             "tiempo_produccion_total": tiempo_total_produccion
#         })

# # Crear DataFrame con los resultados
# df_resultados = pd.DataFrame(resultados)

In [96]:
df_resultados

,maquina,pedido_id,producto,cantidad_producida,prioridad,tiempo_produccion_minutos
0,M1,4,A,42.0,1,100.80
1,M1,16,A,48.0,2,115.20
2,M1,24,A,45.0,3,108.00
3,M1,25,A,48.0,3,115.20
4,M2,1,A,42.0,1,100.80
5,M2,5,B,41.0,2,71.75
6,M2,14,B,46.0,1,80.50
7,M2,40,A,46.0,1,110.40
8,M3,8,B,45.0,3,94.50
9,M3,20,B,41.0,3,86.10


In [99]:
# df_resultados['tiempo_produccion_total_horas'] = df_resultados['tiempo_produccion_total'] / 60

In [100]:
df_resultados.groupby(['maquina']).sum()

,pedido_id,producto,cantidad_producida,prioridad,tiempo_produccion_minutos
maquina,,,,,
M1,69,AAAA,183.0,9,439.20
M2,60,ABBA,175.0,5,363.45
M3,145,BBBBB,295.0,14,619.50
M4,42,BB,105.0,6,220.50


In [66]:
df_orders = pd.read_csv('./orders.csv', index_col='index').reset_index(drop=True)

In [67]:
df_orders

,order_id,product,quantity,priority
0,1,A,42,1
1,2,B,11,2
2,3,B,19,3
3,4,A,42,1
4,5,B,41,2
5,6,B,20,3
6,7,B,33,1
7,8,B,45,3
8,9,B,21,1
9,10,B,38,2


In [102]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpStatus, LpAffineExpression
# Crear el modelo de optimización
modelo = LpProblem("Optimización de Producción con Carga Equitativa", LpMinimize)

# Variables de decisión: asignación de cada pedido a cada máquina
produccion = {}
for _, maquina in df_maquinas.iterrows():
    for _, pedido in df_pedidos.iterrows():
        if not pd.isna(maquina[f'cambio_producto_{pedido["producto"]}']):
            produccion[(maquina["maquina"], pedido["pedido_id"])] = LpVariable(
                f"produccion_{maquina['maquina']}_pedido_{pedido['pedido_id']}", 0, pedido["cantidad"], cat="Integer"
            )

# Crear expresión de tiempo total de trabajo por máquina
tiempo_total_por_maquina = {}
for _, maquina in df_maquinas.iterrows():
    tiempo_total_por_maquina[maquina["maquina"]] = LpAffineExpression(
        lpSum(
            produccion.get((maquina["maquina"], pedido_id), 0) *
            df_maquinas.loc[df_maquinas["maquina"] == maquina["maquina"], f'cambio_producto_{df_pedidos.loc[df_pedidos["pedido_id"] == pedido_id]["producto"].values[0]}'].values[0]
            for pedido_id in df_pedidos["pedido_id"] if (maquina["maquina"], pedido_id) in produccion
        )
    )

# Función objetivo: minimizar el tiempo total de producción ponderado por prioridad y equilibrar el tiempo de las máquinas
objetivo = []
for (maquina, pedido_id), variable in produccion.items():
    pedido = df_pedidos.loc[df_pedidos["pedido_id"] == pedido_id].iloc[0]
    producto = pedido["producto"]
    tiempo_produccion = df_maquinas.loc[df_maquinas["maquina"] == maquina, f'cambio_producto_{producto}'].values[0]
    prioridad = pedido["prioridad"]
    ponderacion_prioridad = 1 / prioridad  # Prioridad alta reduce el tiempo en la función objetivo
    objetivo.append(tiempo_produccion * ponderacion_prioridad * variable)

# Agregar una penalización para balancear el tiempo de las máquinas
tiempo_promedio = lpSum(tiempo_total_por_maquina.values()) / len(df_maquinas)
objetivo_balanceado = lpSum((tiempo_total_por_maquina[maquina] - tiempo_promedio) ** 2 for maquina in tiempo_total_por_maquina)

# Agregar al modelo el objetivo de minimizar tanto el tiempo total ponderado por prioridad como el balance de tiempo de las máquinas
modelo += lpSum(objetivo) + 0.01 * objetivo_balanceado, "Objetivo Total con Balance de Carga"

# Restricciones
# 1. Satisfacer la demanda de cada pedido
for _, pedido in df_pedidos.iterrows():
    pedido_id = pedido["pedido_id"]
    modelo += lpSum(produccion.get((maquina, pedido_id), 0) for maquina in df_maquinas["maquina"]) == pedido["cantidad"], f"demanda_pedido_{pedido_id}"

# 2. Limitar la capacidad máxima de producción por hora para cada máquina
for _, maquina in df_maquinas.iterrows():
    max_capacidad = maquina["capacidad_por_hora"]
    total_piezas_por_hora = lpSum(produccion.get((maquina["maquina"], pedido_id), 0) for pedido_id in df_pedidos["pedido_id"])
    modelo += total_piezas_por_hora <= max_capacidad, f"capacidad_{maquina['maquina']}"

# 3. Limitar el tiempo total de operación de cada máquina a 8 horas (480 minutos)
max_tiempo_maquina = 480  # en minutos
for maquina, tiempo_total in tiempo_total_por_maquina.items():
    modelo += tiempo_total <= max_tiempo_maquina, f"tiempo_total_maquina_{maquina}"

# Resolver el modelo
modelo.solve()

# Crear el DataFrame de resultados
resultados = {
    "maquina": [],
    "pedido_id": [],
    "producto": [],
    "cantidad_producida": [],
    "prioridad": [],
    "tiempo_produccion_minutos": [],
}

for (maquina, pedido_id), variable in produccion.items():
    if variable.varValue > 0:  # Solo considerar asignaciones con producción
        pedido = df_pedidos.loc[df_pedidos["pedido_id"] == pedido_id].iloc[0]
        producto = pedido["producto"]
        cantidad = variable.varValue
        prioridad = pedido["prioridad"]
        tiempo_unitario = df_maquinas.loc[df_maquinas["maquina"] == maquina, f'cambio_producto_{producto}'].values[0]
        tiempo_total = tiempo_unitario * cantidad

        resultados["maquina"].append(maquina)
        resultados["pedido_id"].append(pedido_id)
        resultados["producto"].append(producto)
        resultados["cantidad_producida"].append(cantidad)
        resultados["prioridad"].append(prioridad)
        resultados["tiempo_produccion_minutos"].append(tiempo_total)

df_resultados = pd.DataFrame(resultados)
print(df_resultados)


TypeError: unsupported operand type(s) for ** or pow(): 'LpAffineExpression' and 'int'

--------------

In [2]:
import pandas as pd
df_pedidos = pd.read_csv('./new_orders/orders.csv')

In [3]:
df_pedidos

,index,order_id,product,quantity,priority
0,0,1,A,42,1
1,1,2,B,11,2
2,2,3,B,19,3
3,3,4,A,42,1
4,4,5,B,41,2
5,5,6,B,20,3
6,6,7,B,33,1
7,7,8,B,45,3
8,8,9,B,21,1
9,9,10,B,38,2


In [4]:
import pandas as pd
import numpy as np
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpStatus

# Define machine data
machine_data = {
    "machine": ["M1", "M2", "M3", "M4"],
    "type": ["dual", "dual", "simple", "simple"],
    "capacity_per_hour": [144, 144, 105, 105],  # Capacity in pieces per hour
    "change_product_A": [2.4, 2.4, None, None],  # Production time for product A in minutes
    "change_product_B": [1.75, 1.75, 2.1, 2.1]   # Production time for product B in minutes
}

# Create DataFrame for machines
df_machines = pd.DataFrame(machine_data)

# Generate 50 product orders with random quantities and priorities
df_orders = pd.read_csv('./new_orders/orders.csv')

# Create the optimization model
model = LpProblem("Production Optimization with Multiple Orders", LpMinimize)

# Decision variables: assignment of each order to each machine
production = {}
for _, machine in df_machines.iterrows():
    for _, order in df_orders.iterrows():
        if not pd.isna(machine[f'change_product_{order["product"]}']):
            production[(machine["machine"], order["order_id"])] = LpVariable(
                f"production_{machine['machine']}_order_{order['order_id']}", 0, order["quantity"], cat="Integer"
            )

# Objective function: minimize the total production time weighted by priority and balance workload
objective = []
for (machine, order_id), variable in production.items():
    order = df_orders.loc[df_orders["order_id"] == order_id].iloc[0]
    product = order["product"]
    production_time = df_machines.loc[df_machines["machine"] == machine, f'change_product_{product}'].values[0]
    priority = order["priority"]
    priority_weight = 1 / priority  # Higher priority reduces time in the objective function
    objective.append(production_time * priority_weight * variable)

model += lpSum(objective), "Total Time Weighted by Priority"

# Constraints
# 1. Meet the demand for each order
for _, order in df_orders.iterrows():
    order_id = order["order_id"]
    model += lpSum(production.get((machine, order_id), 0) for machine in df_machines["machine"]) == order["quantity"], f"demand_order_{order_id}"

# 2. Limit maximum production capacity per hour for each machine
for _, machine in df_machines.iterrows():
    max_capacity = machine["capacity_per_hour"]
    total_time = lpSum(production.get((machine["machine"], order_id), 0) for order_id in df_orders["order_id"])
    model += total_time <= max_capacity, f"capacity_{machine['machine']}"

# 3. Limit total operating time of each machine to 8 hours (480 minutes) and balance workload
max_machine_time = 480  # in minutes
for _, machine in df_machines.iterrows():
    total_machine_time = lpSum(production.get((machine["machine"], order_id), 0) *
                               df_machines.loc[df_machines["machine"] == machine["machine"], 
                               f'change_product_{df_orders.loc[df_orders["order_id"] == order_id]["product"].values[0]}'].values[0]
                               for order_id in df_orders["order_id"] 
                               if (machine["machine"], order_id) in production)
    
    model += total_machine_time <= max_machine_time, f"total_time_machine_{machine['machine']}"

# Solve the model
model.solve()

# Display results
print("Assignment of orders to machines:")
for (machine, order_id), variable in production.items():
    if variable.varValue > 0:
        print(f"Machine {machine} produces {variable.varValue} units of Order {order_id}")
print(f"Total Time Weighted by Priority: {model.objective.value()}")
print(f"Optimization Status: {LpStatus[model.status]}")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/silvestf/Documents/freudenberg/venv/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/82ad71e627574520bb11ce13091a723f-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/82ad71e627574520bb11ce13091a723f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 63 COLUMNS
At line 1024 RHS
At line 1083 BOUNDS
At line 1244 ENDATA
Problem MODEL has 58 rows, 160 columns and 480 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Assignment of orders to machines:
Machine M1 produces 42.0 units of Order 4
Machine M1 produces 48.0 units of Order 16
Machine M1 produces 45.0 units of Order 24
Machine M1 produces 48.0 units of Order 25
Machine

/home/silvestf/Documents/freudenberg/venv/lib/python3.10/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [5]:
import pandas as pd

# Collect optimization results into lists
machine_list = []
order_id_list = []
produced_units = []

# Loop through the decision variables and store results where production occurred
for (machine, order_id), variable in production.items():
    if variable.varValue > 0:
        machine_list.append(machine)
        order_id_list.append(order_id)
        produced_units.append(variable.varValue)

# Create a DataFrame to display the assignment of orders to machines
results_df = pd.DataFrame({
    "Machine": machine_list,
    "Order_ID": order_id_list,
    "Produced_Units": produced_units
})

# Show the DataFrame
print(results_df)
print(f"Total Time Weighted by Priority: {model.objective.value()}")
print(f"Optimization Status: {LpStatus[model.status]}")


   Machine  Order_ID  Produced_Units
0       M1         4            42.0
1       M1        16            48.0
2       M1        24            45.0
3       M1        25            48.0
4       M2         1            42.0
5       M2         5            41.0
6       M2        14            46.0
7       M2        40            46.0
8       M3         8            45.0
9       M3        20            41.0
10      M3        23            49.0
11      M3        44            45.0
12      M3        50           115.0
13      M4        11            60.0
14      M4        31            45.0
Total Time Weighted by Priority: 808.175
Optimization Status: Infeasible


In [8]:
import pandas as pd

# Lists to collect results
machine_list = []
order_id_list = []
produced_units = []
production_time = []
weighted_production_time = []

# Loop through decision variables and calculate production time and weighted production time
for (machine, order_id), variable in production.items():
    if variable.varValue > 0:
        # Find product type and priority for the order
        order_info = df_orders.loc[df_orders["order_id"] == order_id].iloc[0]
        product = order_info["product"]
        priority = order_info["priority"]
        
        # Get the time per unit for this machine and product type
        time_per_unit = df_machines.loc[df_machines["machine"] == machine, f'change_product_{product}'].values[0]
        
        # Calculate total production time
        total_time = variable.varValue * time_per_unit  # Produced units * time per unit
        weighted_time = total_time / priority  # Apply priority weighting as per objective

        # Append data to lists
        machine_list.append(machine)
        order_id_list.append(order_id)
        produced_units.append(variable.varValue)
        production_time.append(total_time)
        weighted_production_time.append(weighted_time)

# Create DataFrame with results including weighted production time
results_df = pd.DataFrame({
    "Machine": machine_list,
    "Order_ID": order_id_list,
    "Produced_Units": produced_units,
    "Production_Time_Min": production_time,
    "Weighted_Production_Time": weighted_production_time
})

In [15]:
results_df.groupby('Machine').sum()

,Order_ID,Produced_Units,Production_Time_Min,Weighted_Production_Time
Machine,,,,
M1,69,183.0,439.20,232.800
M2,60,175.0,363.45,327.575
M3,145,295.0,619.50,246.750
M4,42,105.0,220.50,73.500


In [16]:
df_machines

,machine,type,capacity_per_hour,change_product_A,change_product_B
0,M1,dual,144,2.4,1.75
1,M2,dual,144,2.4,1.75
2,M3,simple,105,NaN,2.10
3,M4,simple,105,NaN,2.10


In [17]:
for index, row in df_orders.iterrows():
        row_dict = row.to_dict()

In [21]:
for index, row in df_orders.iterrows():
        row_dict = row.to_dict()
        print(row_dict)

{'index': 0, 'order_id': 1, 'product': 'A', 'quantity': 42, 'priority': 1}
{'index': 1, 'order_id': 2, 'product': 'B', 'quantity': 11, 'priority': 2}
{'index': 2, 'order_id': 3, 'product': 'B', 'quantity': 19, 'priority': 3}
{'index': 3, 'order_id': 4, 'product': 'A', 'quantity': 42, 'priority': 1}
{'index': 4, 'order_id': 5, 'product': 'B', 'quantity': 41, 'priority': 2}
{'index': 5, 'order_id': 6, 'product': 'B', 'quantity': 20, 'priority': 3}
{'index': 6, 'order_id': 7, 'product': 'B', 'quantity': 33, 'priority': 1}
{'index': 7, 'order_id': 8, 'product': 'B', 'quantity': 45, 'priority': 3}
{'index': 8, 'order_id': 9, 'product': 'B', 'quantity': 21, 'priority': 1}
{'index': 9, 'order_id': 10, 'product': 'B', 'quantity': 38, 'priority': 2}
{'index': 10, 'order_id': 11, 'product': 'B', 'quantity': 44, 'priority': 3}
{'index': 11, 'order_id': 12, 'product': 'A', 'quantity': 10, 'priority': 3}
{'index': 12, 'order_id': 13, 'product': 'A', 'quantity': 10, 'priority': 2}
{'index': 13, 'ord

---

### Test Claude use case

In [ ]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, value

# Datos de entrada
maquinas = ["M1", "M2", "M3", "M4"]
productos = ["A", "B"]
capacidad_horaria = [144, 144, 105, 105]  # Capacidad de cada máquina por hora
demandas = {"A": 200, "B": 150}  # Demanda total de cada producto (en unidades)
prioridades = {"A": 3, "B": 1}  # Prioridades de los productos (3 = alta, 1 = baja)

# Modelo de optimización
modelo = LpProblem("Optimizacion_Produccion", LpMinimize)

# Variables de decisión: cantidad de productos asignados a cada máquina
x = LpVariable.dicts("x", (maquinas, productos), lowBound=0, cat="Continuous")

# Variables auxiliares para la carga por máquina
carga_por_maquina = LpVariable.dicts("CargaPorMaquina", maquinas, lowBound=0, cat="Continuous")

# Variables auxiliares para la carga máxima y mínima
carga_max = LpVariable("CargaMax", lowBound=0, cat="Continuous")
carga_min = LpVariable("CargaMin", lowBound=0, cat="Continuous")

# Restricciones: calcular la carga por máquina (reformulación)
for i, m in enumerate(maquinas):
    modelo += carga_por_maquina[m] * capacidad_horaria[i] == lpSum(x[m][p] for p in productos)

# Restricciones de carga máxima y mínima
for m in maquinas:
    modelo += carga_por_maquina[m] <= carga_max
    modelo += carga_por_maquina[m] >= carga_min

# Función objetivo: minimizar la diferencia entre carga máxima y mínima
# y priorizar productos más importantes
modelo += carga_max - carga_min + lpSum(-prioridades[p] * x[m][p] for m in maquinas for p in productos)

# Restricciones de capacidad por máquina
for i, m in enumerate(maquinas):
    modelo += lpSum(x[m][p] for p in productos) <= capacidad_horaria[i] * 8  # Suponiendo jornada de 8 horas

# Restricciones de demanda por producto
for p in productos:
    modelo += lpSum(x[m][p] for m in maquinas) == demandas[p]

# Resolver el modelo
modelo.solve()

# Resultados
print("Resultados de la optimización:")
for m in maquinas:
    for p in productos:
        print(f"Máquina {m}, Producto {p}: {x[m][p].value()} unidades")
    print(f"Carga de la máquina {m}: {value(carga_por_maquina[m])} horas")

print(f"\nCarga máxima: {value(carga_max)} horas")
print(f"Carga mínima: {value(carga_min)} horas")
print(f"Diferencia: {value(carga_max) - value(carga_min)} horas")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/facu/Documents/github/Freudenberg_challenge/venv/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/a3e9c0ca066543c4bd42182cecb86bc6-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/a3e9c0ca066543c4bd42182cecb86bc6-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 63 COLUMNS
At line 271 RHS
At line 330 BOUNDS
At line 347 ENDATA
Problem MODEL has 58 rows, 55 columns and 172 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0.115663 - 0.00 seconds
Cgl0004I processed model has 42 rows, 35 columns (16 integer (0 of which binary)) and 136 elements
Cbc0012I Integer solution of 0.12521825 found by DiveCoefficient after 0 iterations and 0 nodes (0.01 seconds)
Cbc0038I Full problem 42 rows 35 columns, reduced to 30 rows 23 columns
Cbc0031I 8 added rows had aver

In [26]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, value

# Datos de entrada
maquinas = ["M1", "M2", "M3", "M4"]
productos = ["A", "B"]
capacidad_horaria = [144, 144, 105, 105]  # Capacidad de cada máquina por hora
demandas = {"A": 200, "B": 150}  # Demanda total de cada producto (en unidades)
prioridades = {"A": 3, "B": 1}  # Prioridades de los productos (3 = alta, 1 = baja)

# Modelo de optimización
modelo = LpProblem("Optimizacion_Produccion", LpMinimize)

# Variables de decisión: cantidad de productos asignados a cada máquina
x = LpVariable.dicts("x", (maquinas, productos), lowBound=0, cat="Continuous")

# Variables auxiliares para la carga por máquina
carga_por_maquina = LpVariable.dicts("CargaPorMaquina", maquinas, lowBound=0, cat="Continuous")

# Variables auxiliares para la carga máxima y mínima
carga_max = LpVariable("CargaMax", lowBound=0, cat="Continuous")
carga_min = LpVariable("CargaMin", lowBound=0, cat="Continuous")

# Restricciones: calcular la carga por máquina (reformulación)
for i, m in enumerate(maquinas):
    modelo += carga_por_maquina[m] * capacidad_horaria[i] == lpSum(x[m][p] for p in productos)

# Restricciones de carga máxima y mínima
for m in maquinas:
    modelo += carga_por_maquina[m] <= carga_max
    modelo += carga_por_maquina[m] >= carga_min

# Función objetivo: minimizar la diferencia entre carga máxima y mínima
# y priorizar productos más importantes
modelo += carga_max - carga_min + lpSum(-prioridades[p] * x[m][p] for m in maquinas for p in productos)

# Restricciones de capacidad por máquina
for i, m in enumerate(maquinas):
    modelo += lpSum(x[m][p] for p in productos) <= capacidad_horaria[i] * 8  # Suponiendo jornada de 8 horas

# Restricciones de demanda por producto
for p in productos:
    modelo += lpSum(x[m][p] for m in maquinas) == demandas[p]

# Resolver el modelo
modelo.solve()

# Resultados
print("Resultados de la optimización:")
for m in maquinas:
    for p in productos:
        print(f"Máquina {m}, Producto {p}: {x[m][p].value()} unidades")
    print(f"Carga de la máquina {m}: {value(carga_por_maquina[m])} horas")

print(f"\nCarga máxima: {value(carga_max)} horas")
print(f"Carga mínima: {value(carga_min)} horas")
print(f"Diferencia: {value(carga_max) - value(carga_min)} horas")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/facu/Documents/github/Freudenberg_challenge/venv/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/151a333f9cb648e2994bc22a359d48f3-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/151a333f9cb648e2994bc22a359d48f3-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 23 COLUMNS
At line 78 RHS
At line 97 BOUNDS
At line 98 ENDATA
Problem MODEL has 18 rows, 14 columns and 44 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 14 (-4) rows, 10 (-4) columns and 40 (-4) elements
Perturbing problem by 0.001% of 9.989994 - largest nonzero change 0.00018996604 ( 0.2010423%) - largest zero change 0
0  Obj 0 Primal inf 105.10517 (2) Dual inf 52.294102 (9)
9  Obj -749.9839
Optimal - objective value -750
After Postsolve, objective -750, infeasibilities - dual 0 (0), primal 0 (0)
Optimal obj

In [ ]:
import pandas as pd
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, value

# Datos de las máquinas
maquinas = ["M1", "M2", "M3", "M4"]
capacidad_horaria = [144, 144, 105, 105]  # Capacidad de cada máquina por hora

# Cargar los pedidos desde el CSV
archivo_csv = "/home/facu/Documents/github/Freudenberg_challenge/new_orders/orders.csv"  # Reemplaza con la ruta real del archivo
pedidos = pd.read_csv(archivo_csv)

# Agrupar por producto y prioridad
demandas = pedidos.groupby("product")["quantity"].sum().to_dict()
prioridades = pedidos.groupby("product")["priority"].max().to_dict()

# Modelo de optimización
modelo = LpProblem("Optimizacion_Produccion", LpMinimize)

# Variables de decisión: cantidad de productos asignados a cada máquina
productos = list(demandas.keys())
x = LpVariable.dicts("x", (maquinas, productos), lowBound=0, cat="Continuous")

# Variables auxiliares para la carga por máquina
carga_por_maquina = LpVariable.dicts("CargaPorMaquina", maquinas, lowBound=0, cat="Continuous")

# Variables auxiliares para la carga máxima y mínima
carga_max = LpVariable("CargaMax", lowBound=0, cat="Continuous")
carga_min = LpVariable("CargaMin", lowBound=0, cat="Continuous")

# Restricciones: calcular la carga por máquina (reformulación)
for i, m in enumerate(maquinas):
    modelo += carga_por_maquina[m] * capacidad_horaria[i] == lpSum(x[m][p] for p in productos)

# Restricciones de carga máxima y mínima
for m in maquinas:
    modelo += carga_por_maquina[m] <= carga_max
    modelo += carga_por_maquina[m] >= carga_min

# Función objetivo: minimizar la diferencia entre carga máxima y mínima
# y priorizar productos más importantes
modelo += carga_max - carga_min + lpSum(-prioridades[p] * x[m][p] for m in maquinas for p in productos)

# Restricciones de capacidad por máquina
for i, m in enumerate(maquinas):
    modelo += lpSum(x[m][p] for p in productos) <= capacidad_horaria[i] * 8  # Suponiendo jornada de 8 horas

# Restricciones de demanda por producto
for p in productos:
    modelo += lpSum(x[m][p] for m in maquinas) == demandas[p]

# Resolver el modelo
modelo.solve()

# Resultados
print("Resultados de la optimización:")
for m in maquinas:
    for p in productos:
        print(f"Máquina {m}, Producto {p}: {x[m][p].value()} unidades")
    print(f"Carga de la máquina {m}: {value(carga_por_maquina[m])} horas")

print(f"\nCarga máxima: {value(carga_max)} horas")
print(f"Carga mínima: {value(carga_min)} horas")
print(f"Diferencia: {value(carga_max) - value(carga_min)} horas")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/facu/Documents/github/Freudenberg_challenge/venv/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/916297aebba84f72b76b00eefdbc3ae7-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/916297aebba84f72b76b00eefdbc3ae7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 23 COLUMNS
At line 78 RHS
At line 97 BOUNDS
At line 98 ENDATA
Problem MODEL has 18 rows, 14 columns and 44 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 14 (-4) rows, 10 (-4) columns and 40 (-4) elements
Perturbing problem by 0.001% of 9.989994 - largest nonzero change 0.00018996604 ( 0.2010423%) - largest zero change 0
0  Obj 0 Primal inf 424.92518 (2) Dual inf 78.418237 (9)
8  Obj -4244.9259
Optimal - objective value -4245
After Postsolve, objective -4245, infeasibilities - dual 0 (0), primal 0 (0)
Optimal 

In [28]:
import pandas as pd
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, value

# Datos de las máquinas
maquinas = ["M1", "M2", "M3", "M4"]
capacidad_horaria = [144, 144, 105, 105]  # Capacidad de cada máquina por hora
setup_tiempos = {
    "A": [2.1, 2.1, None, None],  # Setup en minutos para producto A
    "B": [1.75, 1.75, 2.1, 2.1]   # Setup en minutos para producto B
}

# Cargar los pedidos desde el CSV
archivo_csv = "/home/facu/Documents/github/Freudenberg_challenge/new_orders/orders.csv"  # Reemplaza con la ruta real del archivo
pedidos = pd.read_csv(archivo_csv)

# Agrupar por producto y prioridad
demandas = pedidos.groupby("product")["quantity"].sum().to_dict()
prioridades = pedidos.groupby("product")["priority"].max().to_dict()

# Modelo de optimización
modelo = LpProblem("Optimizacion_Produccion", LpMinimize)

# Variables de decisión: cantidad de productos asignados a cada máquina
productos = list(demandas.keys())
x = LpVariable.dicts("x", (maquinas, productos), lowBound=0, cat="Continuous")

# Variables auxiliares para la carga por máquina
carga_por_maquina = LpVariable.dicts("CargaPorMaquina", maquinas, lowBound=0, cat="Continuous")

# Variables auxiliares para la carga máxima y mínima
carga_max = LpVariable("CargaMax", lowBound=0, cat="Continuous")
carga_min = LpVariable("CargaMin", lowBound=0, cat="Continuous")

# Restricciones: calcular la carga por máquina
for i, m in enumerate(maquinas):
    modelo += carga_por_maquina[m] * capacidad_horaria[i] == lpSum(x[m][p] for p in productos)

# Restricciones de carga máxima y mínima
for m in maquinas:
    modelo += carga_por_maquina[m] <= carga_max
    modelo += carga_por_maquina[m] >= carga_min

# Función objetivo: minimizar la diferencia entre carga máxima y mínima y priorizar productos importantes
modelo += carga_max - carga_min + lpSum(-prioridades[p] * x[m][p] for m in maquinas for p in productos)

# Restricciones de capacidad por máquina
for i, m in enumerate(maquinas):
    modelo += lpSum(x[m][p] for p in productos) <= capacidad_horaria[i] * 8  # Suponiendo jornada de 8 horas

# Restricciones de demanda por producto
for p in productos:
    modelo += lpSum(x[m][p] for m in maquinas) == demandas[p]

# Resolver el modelo
modelo.solve()

# Procesar resultados para asignar pedidos y calcular tiempos
asignaciones = {m: [] for m in maquinas}  # Pedidos asignados a cada máquina
tiempos_produccion = {m: 0 for m in maquinas}  # Tiempo total por máquina

for m in maquinas:
    for p in productos:
        cantidad = x[m][p].value()
        if cantidad > 0:  # Solo consideramos asignaciones no nulas
            asignaciones[m].append({"producto": p, "cantidad": cantidad, "prioridad": prioridades[p]})

# Ordenar los pedidos en cada máquina por prioridad (mayor primero)
for m in maquinas:
    asignaciones[m].sort(key=lambda pedido: -pedido["prioridad"])

# Calcular tiempos totales (producción y setups)
for i, m in enumerate(maquinas):
    tiempo_total = 0
    producto_anterior = None
    for pedido in asignaciones[m]:
        producto_actual = pedido["producto"]
        cantidad = pedido["cantidad"]

        # Agregar tiempo de producción
        tiempo_produccion = cantidad / capacidad_horaria[i] * 60  # En minutos
        tiempo_total += tiempo_produccion

        # Agregar tiempo de setup si cambia de producto
        if producto_anterior and producto_actual != producto_anterior:
            tiempo_setup = setup_tiempos[producto_actual][i]
            if tiempo_setup:  # Verificar que exista tiempo de setup
                tiempo_total += tiempo_setup

        producto_anterior = producto_actual

    tiempos_produccion[m] = tiempo_total

# Mostrar resultados
print("\nAsignaciones por máquina:")
for m in maquinas:
    print(f"\nMáquina {m}:")
    for pedido in asignaciones[m]:
        print(f"  Producto {pedido['producto']}, Cantidad {pedido['cantidad']}, Prioridad {pedido['prioridad']}")
    print(f"Tiempo total de producción (incluyendo setups): {tiempos_produccion[m]:.2f} minutos")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/facu/Documents/github/Freudenberg_challenge/venv/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/b201f5f76d064d6fa5fc82bdc019bb11-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/b201f5f76d064d6fa5fc82bdc019bb11-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 23 COLUMNS
At line 78 RHS
At line 97 BOUNDS
At line 98 ENDATA
Problem MODEL has 18 rows, 14 columns and 44 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 14 (-4) rows, 10 (-4) columns and 40 (-4) elements
Perturbing problem by 0.001% of 9.989994 - largest nonzero change 0.00018996604 ( 0.2010423%) - largest zero change 0
0  Obj 0 Primal inf 424.92518 (2) Dual inf 78.418237 (9)
8  Obj -4244.9259
Optimal - objective value -4245
After Postsolve, objective -4245, infeasibilities - dual 0 (0), primal 0 (0)
Optimal 

In [29]:
maquinas

['M1', 'M2', 'M3', 'M4']

In [ ]:
ma